In [5]:
import spacy
from scispacy.linking import EntityLinker
from scispacy.abbreviation import AbbreviationDetector
from scispacy.hyponym_detector import HyponymDetector

In [57]:
from HpoFactory import HpoFactory
from HpoLookup import HpoLookup
hpo_F = HpoFactory()
hpo_tree = hpo_F.build_hpo_tree()
hpo_ancestors = hpo_F.get_hpo_ancestors(hpo_tree)
hpo_levels = hpo_F.get_hpo_levels(hpo_tree)
hpo_dict, hpo_name_dict = hpo_F.build_hpo_dict(hpo_ancestors)
hpo_dict = hpo_F.expand_hpo_dict(hpo_dict)

In [6]:
nlp = spacy.load("en_core_sci_sm")
nlp.add_pipe("abbreviation_detector")
nlp.add_pipe("hyponym_detector", last=True, config={"extended": False})
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "hpo"})

/Users/cl3720/Desktop/Doc2Hpo2.0/back-end/.venv/lib/python3.12/site-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]
/Users/cl3720/Desktop/Doc2Hpo2.0/back-end/.venv/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.1.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/cl3720/Desktop/Doc2Hpo2.0/back-end/.venv/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.1.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:

In [7]:
with open('demo_patient_1.txt', 'r') as f:
    text = f.read()

In [8]:
doc = nlp(text)

/Users/cl3720/Desktop/Doc2Hpo2.0/back-end/.venv/lib/python3.12/site-packages/scispacy/abbreviation.py:248: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)


In [86]:
doc.ents

(Age,
 years,
 Female,
 Presentation,
 patient reports,
 persistent,
 fatigue,
 months,
 muscle weakness,
 upper arms,
 thighs,
 morning stiffness,
 minutes,
 difficulty climbing stairs,
 lifting objects,
 Skin changes,
 purple rash,
 eyes,
 heliotrope rash,
 red,
 scaly,
 knuckles,
 Gottron’s papules,
 Mother,
 diagnosed,
 rheumatoid arthritis,
 Preliminary,
 Diagnosis,
 Suspected,
 dermatomyositis,
 pending muscle,
 enzyme studies,
 skin biopsy,
 HPO_terms,
 Urinary urgency,
 Functional abnormality,
 male,
 Hypospadias,
 urinary urgency)

In [79]:
linker = nlp.get_pipe("scispacy_linker")
linker.threshold = 0.7
for entity in doc.ents:
    all_ents = entity._.kb_ents
    if len(entity._.kb_ents) > 0:
        umls_ent = entity._.kb_ents[0]
        hpo_name = linker.kb.cui_to_entity[umls_ent[0]].canonical_name
        hpo_id = hpo_dict[hpo_name] if hpo_name in hpo_dict else None
        if hpo_id is not None:
            print(entity.text,entity.start, entity.end, umls_ent, hpo_name, hpo_id)

persistent 14 15 ('C0476474', 0.7588739395141602) Persistent fever HP:0033399
fatigue 15 16 ('C0015672', 0.9689005017280579) Fatigue HP:0012378
muscle weakness 24 26 ('C0151786', 0.9846422076225281) Muscular weakness HP:0001324
upper arms 28 30 ('C1832117', 0.8244901895523071) Short humerus HP:0005792
morning stiffness 36 38 ('C4025238', 0.8040806651115417) Generalized morning stiffness HP:0005197
difficulty climbing stairs 44 47 ('C0239067', 0.9722282290458679) Difficulty climbing stairs HP:0003551
Skin changes 51 53 ('C1260926', 0.7077376246452332) Abnormality of skin pigmentation HP:0001000
eyes 59 60 ('C0235267', 0.8275376558303833) Red eye HP:0025337
heliotrope rash 61 63 ('C4477048', 0.9855683445930481) Heliotrope rash HP:0040324
scaly 67 68 ('C0237849', 0.7621837258338928) Scaly skin HP:0040189
Gottron’s papules 73 76 ('C0423781', 0.9587185382843018) Gottron's papules HP:0025508
rheumatoid arthritis 85 87 ('C0003873', 0.9730103611946106) RA HP:0001370
Urinary urgency 106 108 ('C

In [70]:
linker.kb.cui_to_entity['C0015672'].canonical_name

'Fatigue'

In [72]:
linker.kb.cui_to_entity[umls_ent[0]].canonical_name

'Urinary urgency'

In [73]:
hpo_dict['Urinary urgency']

'HP:0000012'

In [75]:
[hpo_dict[hpo_name] for hpo_name in hpo_names if hpo_name in hpo_dict]

[]

In [91]:
entity.start_char

666

In [92]:
entity.end_char

681